In [2]:
from libs.llm_chat import (create_chain, check_question, get_session_history, MODEL)
from main import fast_answer

%load_ext autoreload
%autoreload 2

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### История сообщений

In [9]:
session_history = get_session_history('саша320')
a = session_history.messages
a

[HumanMessage(content='Сколько стоит капучино?'),
 AIMessage(content='К сожалению, у меня нет информации о ценах на конкретные продукты в магазинах "Жизньмарт". Однако я могу помочь вам узнать о других наших акциях и товарах! Если вас интересуют еще какие-либо вопросы о нашем магазине, не стесняйтесь спрашивать!'),
 HumanMessage(content='акции на капучино'),
 AIMessage(content='У меня нет актуальной информации о текущих акциях на капучино в магазинах "Жизньмарт". Однако в нашем магазине часто проходят интересные предложения и скидки на различные товары. Рекомендую посетить наш сайт или спросить у сотрудников магазина для получения самой свежей информации. Если у вас есть другие вопросы о "Жизньмарт", я с радостью на них отвечу!')]

In [33]:
from langchain_core.messages import HumanMessage, AIMessage

session_history = get_session_history(user_id)

# Преобразование сообщений в список словарей
messages = []
for message in session_history.messages:
    if isinstance(message, HumanMessage):
        messages.append({"Human": message.content})
    elif isinstance(message, AIMessage):
        messages.append({"AI": message.content})
messages

[{'Human': 'как заказать воду?'},
 {'AI': 'Заказ воды можно оформить в разделе Доставка воды на нашем сайте – https://lifemart.ru/ru/ekb/water-order.'},
 {'Human': 'у меня в салате волос'},
 {'AI': 'Очень жаль слышать о вашей ситуации. Пожалуйста, обратитесь в один из наших магазинов, позвонив по номеру, указанному в разделе «Наши магазины» на сайте или в приложении. Наши супергерои с радостью помогут вам и разберутся в этом вопросе! Не забудьте указать дату изготовления и прикрепить фото, если это возможно. Мы ценим ваше время и хотим, чтобы вы остались довольны!'}]

### Подбор промтов

In [ ]:
from libs.llm_chat import define_hf_chat, HF_KEY


In [14]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from libs.llm_chat import define_promt, get_history_aware_retriever, VEC_STORE_LOAD_PATH

hf_llm = define_hf_chat(api_key=HF_KEY, model='IlyaGusev/saiga_llama3_8b',
                        temperature=0.5, max_tokens=500)
prompt = define_promt(system_prompt)

doc_chain = create_stuff_documents_chain(hf_llm, prompt)
history_aware_retriever = get_history_aware_retriever(hf_llm, VEC_STORE_LOAD_PATH, story_prompt)

chain = create_retrieval_chain(history_aware_retriever, doc_chain)

# Create a chain
conversational_rag_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)
conversational_rag_chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


BadRequestError:  (Request ID: W5OYR05rye0yfPyO7v261)

Bad request:
Authorization header is correct, but the token seems invalid

In [17]:
import requests

API_URL = "https://api-inference.huggingface.co/models/IlyaGusev/saiga_llama3_8b"
headers = {"Authorization": f"Bearer {HF_KEY}"}

# Пример запроса
data = {
    "inputs": "Пример вопроса или текста для модели",
    "parameters": {
        "max_new_tokens": 50,
        "temperature": 0.7
    }
}

response = requests.post(API_URL, headers=headers, json=data)

# Получаем ответ
output = response.json()
print(output)


{'error': 'Authorization header is correct, but the token seems invalid'}


### Подключаем GPT


In [7]:

from libs.llm_chat import OPENAI_KEY

story_prompt = """find the documents closest to the question in meaning
    """
# """Далее представлена история сообщений данного гостя с чат-ботом. В ней может не быть правильных ответов,
# ведь если бы человек получил правильный ответ, он бы больше не писал боту.
# Но в ней может быть полезный для поиска правильного ответа контекст.
#     """
system_prompt = """Ты - чат-бот консультант, и работаешь в чате службы поддержки сети магазинов хороших продуктов "Жизньмарт",
    твоя функция - стараться ответить на любой вопрос клиента про работу магазинов "Жизьмарт".
    Ты говоришь только на чистом, грамотном русском языке без ошибок!
    Если вопрос не касается контекста, то вежливо и дружелюбно расскажи про Жизьмарт.

    {context}

    Используй только этот контекст, чтобы ответить на последний вопрос.
    Твой ответ должен быть полным и точно соответствовать тому, что написано в context.
    Если ответа нет в контексте, просто позитивно поддержи диалог на тему Жизньмарта!
    """
# Сформулируй ответ на основе того, как человек общается с ботом в context
# если ответа на вопрос, касающийся работы магазинов Жизньмарт нет в context, ответь, что пока не готов ответить на этот вопрос, т.к. он находится в рассмотрении

chain = create_chain(model=MODEL, api_key=OPENAI_KEY,
                     system_prompt=system_prompt, story_prompt=story_prompt,
                     temperature=1, max_tokens=500)
chain

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F608E75600>))], default=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='find the documents closest to the question in meaning\n    ')), Hu

In [4]:
user_id = 'саша333'
question = 'к вам можно с животными?'

In [8]:
def tm(question):
    question = check_question(question)
    if fast_answer(question):
        return fast_answer(question)
    ans_dict = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
    # ans_dict['answer'] = ans_dict['answer'].replace('\n', '')
    return ans_dict
tm(question)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


AssertionError: 

In [70]:
def tm(question):
    question = check_question(question)
    if fast_answer(question):
        return fast_answer(question)
    ans_dict = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
    ans_dict['answer'] = ans_dict['answer'].replace('\n', '')
    return ans_dict
tm(question)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'А с собачками к вам можно?',
 'chat_history': [],
 'context': [Document(page_content='Человек:к вам можно с собаками? Пустите с пёсиком в магазин?  \nБот: Да, мы pet -friedly и любим, когда вы приходите к нам в гости с вашими питомцами!'),
  Document(page_content='Человек: Почему не отображается статус заказа в личном кабинете?  \nБот: Попробуйте свернуть приложение и вновь его о ткрыть. Перезагрузка обычно \nпомогает 💚'),
  Document(page_content='Человек: Как рассчитывается стоимость доставки?  \nБот: Стоимость доставки рассчитывается автоматически в зависимости от адреса и \nвремени заказа.  В пешей зоне (до 1 км от магазина) доставка бесплатная  с 9:00 до \n21:00.  Если адрес доставки удален от магазина, то заказ будет доставлен оператором \nтакси от двери до двери. Стоимость такси посчитает перевозчик. А наша умная \nсистема подберет перевозчика с самым выгодным предложением.'),
  Document(page_content='Человек: Как связаться с курьером?  \nБот: В приложении в разделе «Н

### БЕСПЛАТНЫЕ МОДЕЛЬКИ

In [2]:
story_prompt = """Учитывая историю чата и последний вопрос пользователя,
    который может ссылаться на контекст в истории чата, сформулируй отдельный вопрос,
    который можно понять без истории чата. НЕ ОТВЕЧАЙ на вопрос, просто переформулируйте его,
    если необходимо, и в противном случае верните его как есть.
    """
# """Далее представлена история сообщений данного гостя с чат-ботом. В ней может не быть правильных ответов,
# ведь если бы человек получил правильный ответ, он бы больше не писал боту.
# Но в ней может быть полезный для поиска правильного ответа контекст.
#     """
system_prompt = """Ты - чат-бот консультант, и работаешь в чате службы поддержки сети магазинов хороших продуктов "Жизньмарт",
    твоя функция - стараться ответить на любой вопрос клиента про работу магазинов "Жизьмарт".
    Ты говоришь только на чистом, грамотном русском языке без ошибок!
    Если вопрос не касается контекста, то вежливо и дружелюбно расскажи про Жизьмарт.

    {context}

    Используй только этот контекст, чтобы ответить на последний вопрос.
    Твой ответ должен быть полным и точно соответствовать тому, что написано в context.
    Если ответа нет в контексте, просто позитивно поддержи диалог на тему Жизньмарта!
     """
# Сформулируй ответ на основе того, как человек общается с ботом в context
# если ответа на вопрос, касающийся работы магазинов Жизньмарт нет в context, ответь, что пока не готов ответить на этот вопрос, т.к. он находится в рассмотрении

chain = create_chain(model=qwen2,
                     system_prompt=system_prompt, story_prompt=story_prompt,
                     temperature=0.5, max_tokens=500)
chain

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBranch(branches=[(RunnableBinding(bound=RunnableLambda(_is_not_async), config={'run_name': 'RunnableWithMessageHistoryInAsyncMode'}), RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D55572B880>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, lang

In [4]:
def tm(question):
    question = check_question(question)
    if fast_answer(question):
        return fast_answer(question)
    ans_dict = chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}}) 
    ans_dict['answer'] = ans_dict['answer'].replace('\n', '')
    return ans_dict
tm(question)

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainPendingDeprecationWarning: This class is deprecated and will be removed in a future version. You can swap to using the `PostgresChatMessageHistory` implementation in `langchain_postgres`. Please do not submit further PRs to this class.See <https://github.com/langchain-ai/langchain-postgres>
  warn_deprecated(
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'заказа нет час никто не берет телефон',
 'chat_history': [],
 'context': [Document(page_content='Человек: Как сделать заказ?  \nБот: Оформить заказ можно в нашем приложении или н а сайте, добавив все нужное в \nкорзину 💚'),
  Document(page_content='Человек: Почему не отображается статус заказа в личном кабинете?  \nБот: Попробуйте свернуть приложение и вновь его о ткрыть. Перезагрузка обычно \nпомогает 💚'),
  Document(page_content='Человек : Где мой заказ?  \nБот: Добрый день! Приносим тысячу извинений, что задерживаемся. Уточнить \nместоположение курьера можно напрямую в магазине – после оформления заказа \nпоявляется кнопка "Связаться с нами" – это прямой номер филиала.'),
  Document(page_content='Человек: Отменили заказ а денег нет  \nБот: Ден ежные средства за отмененный заказ возвращаются на счет в зависимости от \nусловий вашего банка – регламент от 1 до 30 рабочих дней, однако на практике \nвозврат приходит в течение нескольких дней. Пожалуйста, ожидайте')],
 'answer'

In [39]:
chain.invoke({"input": question}, config={"configurable": {"session_id": user_id}})

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [HumanMessage(content='Курьеранет сколько ждать еще?'),
  AIMessage(content='Извините, я не могу отследить местоположение курьера. Но если у вас есть номер телефона курьера, вы можете позвонить ему и уточнить.')],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Как сменить данные в профиле?  \nБот: Здравствуйте! Для этого перейдите в профиль в приложении или на сайте в \nЛичный Кабинет и внесите нужные измен

In [16]:
import uuid

user_id = 'саша102'
# user_id = str(uuid.uuid4())
question = 'Курьеранет сколько ждать еще?'
# user_id

In [15]:
response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Какой кофе мне могут сделать бесплатно по карточке?  \nБот: Здравствуйте! На главном экране в приложен ии Жизньмарт  появился раздел \nКофе в подарок, в котором есть копилка. Нажав на акцию с кофе, вы увидите, какие \nнапитки можно получить бесплатно 💚'),
  Document(page_content='Гость: Спасибо, понятно/ До свидания/ решение проблемы и завершение диалога  \

In [17]:
response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Как связаться с курьером?',
 'chat_history': [HumanMessage(content='Курьеранет сколько ждать еще?'),
  AIMessage(content='Здравствуйте! К сожалению, я не могу знать, когда именно приедет курьер. Но вы всегда можете отследить его местоположение в приложении или позвонить в магазин, чтобы уточнить информацию.')],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как сдела ть заказ?  \nБот: Здравствуйте! 💚 Оформить заказ можно в нашем приложении или на сайте, \nдобавив все нужное в корзину.'),
  Document(page_content='Гость: Где мой заказ?  \nБот: Добрый день! Приносим тысячу извинений, что задерживаемся. Уточнить \nместоположение курьера можно напрямую в магазине – после оформления зак

In [5]:
user_id = 'саша103'
question = 'Курьеранет сколько ждать еще?'

response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

C:\D\livemart\chat_bot_lm\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainPendingDeprecationWarning: This class is deprecated and will be removed in a future version. You can swap to using the `PostgresChatMessageHistory` implementation in `langchain_postgres`. Please do not submit further PRs to this class.See <https://github.com/langchain-ai/langchain-postgres>
  warn_deprecated(
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Как связаться с курьером?',
 'chat_history': [],
 'context': [Document(page_content='Гость: Как связаться с курьером?  \nБот: Здравствуйте! В приложении в разделе «Наши магазины» и на сайте в разделе \n«Адреса магазинов кафе» представлены все номера магазинов – вы можете \nпозвонить напрямую супергероям, а они сориентируют вас по коммуникации с \nкурьером :)'),
  Document(page_content='Гость: Как оставить чаевые курьеру?  \nБот: Здравствуйте! После завершения  заказа есть кнопка оставить чаевые курьеру, \nгде вы указываете нужную сумму и переводите благодарность нашему супергерою! :)'),
  Document(page_content='Гость: Как отменить заказ?  \nБот: Здравствуйте! После оформления заказа есть активная кнопка "Связаться с нами" \n– вы увидите номер магазина, по которому можете обратиться напрямую к \nсупергероям и попросить отменить заказ :)'),
  Document(page_content='Гость: Как изменить магазин для доставки?  \nБот: Здравствуйте! Магазин вручную выбрать нельзя. Приложение автома

In [6]:
user_id = 'саша103'
question = 'Курьеранет сколько ждать еще?'

response_content = chain_gigachat.invoke({"input": question}, config={"configurable": {"session_id": user_id}})
response_content

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'Курьеранет сколько ждать еще?',
 'chat_history': [HumanMessage(content='Как связаться с курьером?'),
  AIMessage(content='Здравствуйте! Вы можете связаться с курьером, позвонив напрямую в магазин "Жизньмарт". В приложении или на сайте есть раздел "Наши магазины", где указаны все номера телефонов.')],
 'context': [Document(page_content='Гость: Где можно оставить отзыв?  \nБот: Здравствуйте! Отзыв можем принять в Живом чате – просто отправьте слово \n"оператор", а затем опишите ваш отзыв, а также укажите магазин, где вы делали \nпокупку, и наш оператор поможет вам :)'),
  Document(page_content='Гость: Не могу оплатить продукцию живчик ами. \nБот: Здравствуйте! :) Живчики можно обменять на любой товар из Живчикмарт, \nдобавив все нужное в корзину приложения и оформив заказ на доставку или \nсамовывоз (магазин живчиков находится в главном меню в приложении).'),
  Document(page_content='Гость: Почему в магазине нет нужного товара  \nБот: Здравствуйте! Ежедневно мы принимаем поста

Комментарий по ответу:

'answer': 'Здравствуйте! К сожалению, я не могу точно сказать, сколько времени потребуется курьеру, чтобы доставить ваш заказ. Однако, если у вас возникли вопросы по поводу доставки, вы всегда можете обратиться в службу поддержки клиентов магазина "Жизньмарт".'

Ты уже чат-бот службы поддержки клиентов сети магазинов "Жизньмарт"! Куда ты направляешь людей, они уже к тебе обратились. И зачем ставить точку в конце каждого предложения? Это пассивно-агрессивно...
Нужно менять системные промты